In [2]:
import os
import rasterio
import numpy as np
from tqdm import tqdm
from rasterio.merge import merge
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import from_origin, from_bounds

In [9]:
import rasterio
import numpy as np

def calculate_ndsm(dsm_path, dtm_path, output_path):
    # Open DSM and DTM raster files
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(dtm_path) as dtm_src:
        # Read DSM and DTM data as NumPy arrays
        dsm = dsm_src.read(1)
        dtm = dtm_src.read(1)

        # Identify NoData areas in both DSM and DTM
        nodata_dsm = (dsm == dsm_src.nodata)
        nodata_dtm = (dtm == dtm_src.nodata)

        # Mask NoData values in both DSM and DTM
        dsm = np.ma.masked_array(dsm, mask=nodata_dsm)
        dtm = np.ma.masked_array(dtm, mask=nodata_dtm)

        # Calculate nDSM only where both DSM and DTM have valid data
        ndsm = dsm - dtm

        # Normalize nDSM by a scaling factor (e.g., 5.0)
        scaling_factor = 5.0
        ndsm_normalized = np.ma.masked_array(ndsm * scaling_factor, mask=(nodata_dsm | nodata_dtm))
        ndsm_normalized[ndsm_normalized < 0] = 0
        ndsm_normalized[ndsm_normalized > 255] = 255
        ndsm_normalized = ndsm_normalized.astype(np.uint8)
        
        # Write the nDSM to a new GeoTIFF file
        profile = dsm_src.profile.copy()
        profile.update(count=1, dtype=rasterio.uint8, nodata=None)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(np.ma.filled(ndsm_normalized, fill_value=0), 1)

if __name__ == "__main__":
    # Replace with the actual file paths
    dsm_file_path = "/mnt/Data2/sli/dataset/img/italy/fe_dsm_merged_cog.tif"
    dtm_file_path = "/mnt/Data2/sli/dataset/img/italy/fe_dtm_merged_cog.tif"
    output_ndsm_path = "/mnt/Data2/sli/dataset/img/italy/fe_ndsm_merged.tif"

    calculate_ndsm(dsm_file_path, dtm_file_path, output_ndsm_path)


In [8]:
import os
import pyproj
import rasterio
import numpy as np
from scipy import interpolate
from rasterio.enums import Resampling

def upsample_2d_array(input_array, target_resolution):
    """
    Upsample a 2D array to a given resolution using bilinear interpolation.

    Parameters:
    - input_array: 2D NumPy array to be upsampled.
    - target_resolution: Tuple (new_rows, new_columns) specifying the target resolution.

    Returns:
    - upsampled_array: 2D NumPy array after upsampling.
    """
    rows, cols = input_array.shape
    target_rows, target_cols = target_resolution

    # Create grids for the original and target resolutions
    x = np.arange(0, cols)
    y = np.arange(0, rows)
    x_new = np.linspace(0, cols - 1, target_cols)
    y_new = np.linspace(0, rows - 1, target_rows)

    # Create 2D interpolation function using bilinear interpolation
    interpolator = interpolate.RectBivariateSpline(y, x, input_array, kx=1, ky=1)

    # Perform interpolation on the new grids
    upsampled_array = interpolator(y_new, x_new).astype('uint8')

    return upsampled_array


def transform_bbox_epsg_22232_to_7791(bbox):
    # Define the source and destination projections
    transformer = pyproj.Transformer.from_crs("EPSG:22232", "EPSG:7791")

    # Transform the bounding box coordinates
    lon1, lat1, lon2, lat2 = bbox
    x1, y1 = transformer.transform(lon1, lat1)
    x2, y2 = transformer.transform(lon2, lat2)

    # Return the transformed bounding box coordinates
    transformed_bbox = [x1, y1, x2, y2]
    return transformed_bbox

def process_and_concatenate(input_folder, output_folder, merged_ndsm_path, shapefile_path):
                                
    # List all 4-channel GeoTIFF files in the input folder
    tiff_files = [file for file in os.listdir(input_folder) if file.endswith(".tif")]
        
    for tiff_file in tqdm(tiff_files):
        tiff_path = os.path.join(input_folder, tiff_file)
        output_path = os.path.join(output_folder, f"{tiff_file}")

        # Get the spatial extent of the 4-channel GeoTIFF
        with rasterio.open(tiff_path) as tiff_src:
            bounds = tiff_src.bounds
            bounds = transform_bbox_epsg_22232_to_7791(bounds)
        
        with rasterio.open(merged_ndsm_path) as ndsm_src:
            # Calculate nDSM within the specified bounds        
            dst_window = rasterio.windows.from_bounds(*bounds, ndsm_src.transform)
            seg_ndsm = ndsm_src.read(window=dst_window, boundless=True, out_dtype='uint8', fill_value=0)
        
        seg_ndsm = upsample_2d_array(np.squeeze(seg_ndsm), (tiff_src.height, tiff_src.width))

        # Open the 4-channel GeoTIFF file
        with rasterio.open(tiff_path) as tiff_src:
            # Read the existing 4 channels
            channels = [tiff_src.read(i) for i in range(1, 5)]

            # Append the nDSM as the fifth channel
            channels.append(seg_ndsm)

            # Update the profile to include the fifth channel
            profile = tiff_src.profile.copy()
            profile.update(count=5, dtype=rasterio.uint8)

            # Write the new 5-channel GeoTIFF file
            with rasterio.open(output_path, 'w', **profile) as dst:
                for i, channel_data in enumerate(channels, start=1):
                    dst.write(channel_data, i)



input_folder = "/mnt/Data2/sli/dataset/img/italy/Ferrara_ortofoto2022_10_cm/"
output_folder = "/mnt/Data2/sli/dataset/img/italy/rgbin/"
shapefile_path = '/mnt/Data2/sli/dataset/img/italy/italy_bounds.shp'
merged_ndsm_path = "/mnt/Data2/sli/dataset/img/italy/fe_ndsm_merged.tif"

process_and_concatenate(input_folder, output_folder, merged_ndsm_path, shapefile_path)


100%|██████████| 26/26 [56:26<00:00, 130.23s/it]
